In [1]:
from scripts.GetSatellitesGSM import get_artemis_gsm
from scipy.stats import linregress, pearsonr
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import pandas as pd
import numpy as np
import os

In [24]:
'''
This cell  imports a list of ICMEs by Richardson and Cane and compares them to a list of times when ARTEMIS is on the E-S line
'''

icme_list = pd.read_csv('lists/icme-list.csv', header=0, parse_dates=[0, 1, 2], date_format='%Y/%m/%d %H%M')    # Import the list of ICMEs observed by OMNI
artemis_dates = pd.read_csv('../artemis-near-ESline.csv', header=0,  parse_dates=[0, 1]) # Import times when ARTEMIS was near the E-S line

valid_starts = []
valid_stops = []
# This loop iterates through this list of starts and stops when ARTEMIS is on the E-S line and compares them to the intervals of HSSs

for astart, astop in zip(artemis_dates['Date_Start'], artemis_dates['Date_Stop']):
    for hstart, hstop in zip(icme_list['ICME Start (UT)'], icme_list['ICME Stop (UT)']):
        if (astart <= hstop) and (astop >= hstart): # Check if intervals overlap
            valid_start = max(astart, hstart)   # Find the latter of the two start times
            valid_stop = min(astop, hstop)  # Find the earlier of the two end times

            valid_starts.append(dt.datetime.strftime(valid_start, format='%Y-%m-%d %H:%M:%S'))
            valid_stops.append(dt.datetime.strftime(valid_stop, format='%Y-%m-%d %H:%M:%S'))

            fig, ax = plt.subplots(figsize=(15, 2))
            ax.barh(0.7, (hstop - hstart), left=hstart, height=0.4, color='red', alpha=0.6, label='ICME')
            ax.barh(0.3, (astop - astart), left=astart, height=0.4, color='blue', alpha=0.6, label='ARTEMIS')

            ax.set_title(f'Event {dt.datetime.strftime(valid_start, format="%Y-%m-%d")}')
            ax.set_yticks([])  # Remove y-axis ticks since they're not meaningful
            ax.set_xlabel('Date and Time (UTC)')
            ax.grid(True, alpha=0.3)
            ax.legend(loc='upper left')

            # Format x-axis to show dates nicely
            #date_formatter = mdates.DateFormatter('%Y-%m-%d %H:%M')
            #ax.xaxis.set_major_formatter(date_formatter)

            ax.xaxis.set_major_locator(mdates.HourLocator(interval=6))
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d %H:%M'))
            ax.xaxis.set_minor_locator(mdates.HourLocator(interval=1))
            #plt.gcf().autofmt_xdate()

            # Adjust layout to prevent date labels from being cut off
            plt.tight_layout()

            plt.savefig(f'lists/overlap_plots/event_{dt.datetime.strftime(valid_start, format="%Y-%m-%d")}.png', bbox_inches='tight', dpi=300)
            plt.close()

intervals = pd.DataFrame({'ARTEMIS_icme_start': valid_starts, 'ARTEMIS_icme_stop': valid_stops})  # Create a DataFrame with all the valid starts and stops
intervals.to_csv('lists/ICME_intervals.csv', index=None) # Convert the dataframe to a CSV